In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

import sys
sys.path.append('../src')
from data_imports import *

In [2]:
BIOSAMPLES = import_biosamples()

In [8]:
def import_cbtn_TP53():
    path="data/pbta_TP53_alterations.xlsx"
    df = pd.read_excel(path)
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    # aggregate to patient level
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_cbtn_TP53_pathway():
    path="data/alterations_across_samples.tsv"
    df = pd.read_csv(path,sep='\t')
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_hgg_bs():
    hgg_bs = import_biosamples()
    hgg_bs = hgg_bs[(hgg_bs.cancer_type == "HGG") & (hgg_bs.in_unique_patient_set)]
    return hgg_bs
def import_hgg_pt():
    hgg_pt = import_patients()
    hgg_pt = hgg_pt[(hgg_pt.cancer_type == "HGG")]
    hgg_pt = hgg_pt.merge(import_cbtn_TP53_pathway(),how='inner',left_index=True,right_index=True)
    return hgg_pt



In [12]:
pt = import_hgg_pt()
contingency_tbl = pd.crosstab(pt.amplicon_class == "ecDNA",pt.TP53_altered)
print(
scipy.stats.chi2_contingency(contingency_tbl)
)
contingency_tbl

Chi2ContingencyResult(statistic=np.float64(5.635625373803828), pvalue=np.float64(0.017599066269750028), dof=1, expected_freq=array([[ 69.66666667, 139.33333333],
       [ 21.33333333,  42.66666667]]))


TP53_altered,False,True
amplicon_class,,
False,78,131
True,13,51
